# SDCA

On commence par mettre en place un algorithme SDCA classique

### Génération des données

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(0)

n_samples, n_features = 1000, 5
X = np.random.randn(n_samples, n_features)
w_real = np.random.randn(n_features)
y = np.sign(X.dot(w_real) + np.random.randn(n_samples))

### Définition des fonctions de perte

In [38]:
# On définit la fonction qui update alpha pour la hinge loss
def update_hinge_loss(y, w, x, n, lamb, alpha, gamma=0):
    """
        Update alpha pour la hinge loss
        y: label
        w: poids actuels
        x: inputs
        n: nombre de samples
        lamb: step
        alpha: variable du dual
        gamma: smoothness
    """
    minim = min(1, ((1-x.dot(w)*y)/(x.dot(x)/(lamb*n)))+alpha*y)
    delta_alpha = y* max(0, minim)-alpha
    return (delta_alpha)     
    
# On définit la fonction qui update alpha pour la absolute deviation loss 
def update_absolute_deviation_loss(y, w, x, n, lamb, alpha, gamma=0):
    """
        Update alpha pour la absolute deviation loss
        y: label
        w: poids actuels
        x: inputs
        n: nombre de samples
        lamb: step
        alpha: variable du dual
        gamma: smoothness
    """
    minim = min(1, ((y-x.dot(w))/(x.dot(x)/(lamb*n)))+alpha)
    delta_alpha = max(-1, minim)-alpha
    return (delta_alpha)

# On définit la fonction qui update alpha pour la smooth hinge loss 
def update_smoothed_hinge_loss(y, w, x, n, lamb, alpha, gamma):
    """
        Update alpha pour la smoothed hinge loss 
        y: label
        w: poids actuels
        x: inputs
        n: nombre de samples
        lamb: step
        alpha: variable du dual
        gamma: smoothness
    """
    minim = min(1, ((1-x.dot(w)*y-gamma*alpha*y)/(x.dot(x)/(lamb*n+gamma)))+alpha*y)
    delta_alpha = y*max(0, minim)-alpha
    return (delta_alpha)

### Définition de l'algorithme

In [43]:
def SDCA (X, y, T, update_loss, gamma, T_0_ratio, lamb = 1e-5 ):
    """
        Algorithme SDCA classique
        X: variables explicatives
        y: label a prdire
        T: nombre d'epoch
        update_loss: choix de la loss function qui influe l'update de alpha
        gamma: parametre de smoothing, 1 etant tres smooth
        T_0_ratio: fraction qui indique quelle part des iterations on conserve
                    pour moyenner et obtenir alpha et w finaux
        lamb: step
    """
   
    n_samples = len(y)
    n_features=X.shape[1]
    T = T*n_samples
    T_0 = int(T * T_0_ratio)
    
    #Initialisations
    W=np.zeros(n_features*T_0).reshape(T_0,n_features)
    Alpha = np.zeros(n_samples*T_0).reshape(T_0,n_samples)

    #On fait tourner les premieres iterations sans sauvegarder les resultats

    for t in range (0,T-T_0):
        i = np.random.random_integers(0,n_samples-1)
        delta_alpha = update_loss(y[i], W[0], X[i], n_samples, lamb, Alpha[0][i], gamma)

        # On update alpha
        Alpha[0] = Alpha[0] 
        # On update la coordonnée de alpha qui correspond à l'observation choisie
        Alpha[0][i]+=delta_alpha

        # On update w
        W[0] = W[0]+ (X[i].dot(delta_alpha))

    #Puis on sauvegarde le reste

    for t in range (0,T_0-1):
        i = np.random.random_integers(0,n_samples-1)
        delta_alpha = update_loss(y[i], W[t], X[i], n_samples, lamb, Alpha[t][i], gamma)

        # On update alpha
        Alpha[t+1] = Alpha[t] 
        # On update la coordonnée de alpha qui correspond à l'observation choisie
        Alpha[t+1][i]+=delta_alpha

        # On update w
        W[t+1] = W[t]+ (X[i].dot(delta_alpha))

    # Sortie en moyenne
    alpha_bar = Alpha.mean(axis=0)
    w_bar = W.mean(axis=0)
    print("true weights", w_real)
    print("-------------------------------------")
    print("w bar",w_bar)
    print("squared norm of difference",(w_real-w_bar).dot(w_real-w_bar))
    
    # Sortie aleatoire
    j = np.random.random_integers(0,T_0-1)
    alpha_rand = Alpha[j]
    w_rand=W[j]
    print("-------------------------------------")
    print("w random",w_rand)
    print("squared norm of difference",(w_real-w_rand).dot(w_real-w_rand))

### On fait tourner l'algorithme

In [45]:
#Hyperparametres
SDCA(X, y, 10, update_smoothed_hinge_loss, 1.0 , 1/2, 1e-6)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: This function is deprecated. Please call randint(0, 999 + 1) instead
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: This function is deprecated. Please call randint(0, 999 + 1) instead


true weights [ 0.30972382 -0.73745619 -1.53691988 -0.56225483 -1.59951112]
-------------------------------------
w bar [ 0.19303797 -0.51860118 -1.16337421 -0.4275179  -1.25693034]
squared norm of difference 0.336565111085642
-------------------------------------
w random [ 0.14624221 -0.58040864 -0.9776572  -0.55609604 -1.52462582]
squared norm of difference 0.36981066425058257


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:64: DeprecationWarning: This function is deprecated. Please call randint(0, 4999 + 1) instead
